# 🔌 Réseau Électrique Européen : Analyse de Coupure de Ligne

Ce notebook démontre l'**analyse de contingence N-1** sur le réseau électrique européen en utilisant la **méthode de Lanczos/Hamiltonien** pour l'approximation des flux de puissance.

## Sommaire

1. **Introduction du Réseau** : Visualisation du réseau de transport haute tension européen avec distribution des flux
2. **Étude de Cas 1** : Coupure de la ligne Lyon-Grenoble (Ligne 378)
3. **Étude de Cas 2** : Coupure de l'interconnexion transfrontalière Perpignan-Gérone (Ligne 340)

## Méthodologie

L'analyse utilise le **principe de superposition** :

$$\psi_{après} = \psi_{avant} - I_{coupé} \times \psi_{dipôle}$$

Où :
- $\psi_{avant}$ = distribution des flux de puissance de référence
- $I_{coupé}$ = courant traversant la ligne coupée avant la déconnexion  
- $\psi_{dipôle}$ = motif de flux d'une injection dipôle unitaire aux extrémités de la ligne

## Comment fonctionne l'algorithme de Lanczos sans bus d'injection/extraction explicites ?

Contrairement à l'approche classique avec un seul point d'injection (+1) et d'extraction (-1), ici nous utilisons **tous les générateurs et charges du réseau** comme vecteur initial $q_1$ :

1. **Initialisation** : Le vecteur $q_1$ contient la puissance nette de chaque bus (génération - charge), normalisé à norme unitaire
2. **Itérations de Lanczos** : L'algorithme construit un sous-espace de Krylov en multipliant successivement par la matrice Hamiltonienne $H$ (matrice d'adjacence pondérée par les susceptances)
3. **Approximation de ψ** : Le potentiel est approximé par $\psi \approx \sum_i \kappa_{2i} \cdot q_{2i}$ où les $\kappa_i$ sont calculés à partir des coefficients $\beta_i$

Cette approche **distribue naturellement** les injections/extractions sur tout le réseau selon les données réelles de production et consommation !

In [ ]:
# ===========================================
# IMPORTS AND CONFIGURATION
# ===========================================

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pypsa
import pandas as pd
import plotly.graph_objects as go
import copy
import warnings
warnings.filterwarnings('ignore')

from utils import EuropeanGrid

# Plotting configuration
plt.rc('font', family='arial', weight='bold', size=12)

In [ ]:
# ===========================================
# LOAD PYPSA NETWORK
# ===========================================

n = pypsa.Network("./networks/elec_s_512.nc")

print(f"✓ Network loaded: {len(n.buses)} buses, {len(n.lines)} lines")
print(f"  Countries: {n.buses['country'].unique()}")

In [ ]:
# ===========================================
# BUILD EUROPEAN GRID WITH HAMILTONIAN METHOD
# ===========================================

# Create grid using real power data (all generators/loads)
grid = EuropeanGrid(
    pypsa_network=n, 
    q_N=len(n.buses) * 2,  # Number of Lanczos iterations
    use_real_power=True,   # Use actual generator/load data
    real_data=True         # Use actual line lengths for susceptance
)

grid.build_from_pypsa()

# Filter to largest connected component
""" largest_cc = max(nx.connected_components(grid), key=len)
nodes_to_remove = set(grid.nodes()) - largest_cc
grid.remove_nodes_from(nodes_to_remove) """

# Update internal lists
grid._nodes = [n for n in grid._nodes if n in grid.nodes()]
grid._lines = [l for l in grid._lines if l in grid.nodes()]

# Recalculate bus_power for remaining nodes
remaining_buses = [str(n).replace('N_', '') for n in grid._nodes]
grid.bus_power = {k: v for k, v in grid.bus_power.items() if k in remaining_buses}

# Re-balance power (ensure sum = 0)
total_power = sum(grid.bus_power.values())
mean_adjustment = total_power / len(grid.bus_power)
for bus_id in grid.bus_power:
    grid.bus_power[bus_id] -= mean_adjustment

print(f"✓ Grid prepared: {len(grid._nodes)} bus nodes, {len(grid._lines)} line nodes")
print(f"  Power balance: {sum(grid.bus_power.values()):.6f} MW (should be ~0)")

In [ ]:
import json
import networkx as nx


def add_submarine_cables_with_midpoint(G, json_file):
    """
    Ajoute les câbles et calcule la position du nœud de ligne au milieu des deux bus.
    """
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
            cables = data.get('submarine_cables', [])
    except FileNotFoundError:
        print(f"❌ Erreur : Le fichier {json_file} est introuvable.")
        return
    added = 0
    for cable in cables:
        u, v = cable["from_node"], cable["to_node"]
        
        if u in G.nodes() and v in G.nodes():
            # 1. Récupération des positions des bus connectés
            pos_u = G.nodes[u].get('pos')
            pos_v = G.nodes[v].get('pos')
            
            if pos_u and pos_v:
                # 2. Calcul du milieu : (x1+x2)/2 , (y1+y2)/2
                mid_x = (pos_u[0] + pos_v[0]) / 2
                mid_y = (pos_u[1] + pos_v[1]) / 2
                mid_pos = (mid_x, mid_y)
                
                # 3. Création du nœud de ligne
                line_id = f"L_Subsea_{cable['name'].replace(' ', '_')}"
                G.add_node(line_id, 
                           type='line', 
                           pos=mid_pos, 
                           weight=cable['capacity_mw'], 
                           is_submarine=True)
                
                # 4. Connexions biparties
                G.add_edge(u, line_id)
                G.add_edge(line_id, v)
                
                # Mise à jour de la liste interne des lignes
                if hasattr(G, '_lines') and line_id not in G._lines:
                    G._lines.append(line_id)
                
                added += 1
                print(f"✅ {cable['name']} ajouté au milieu : {mid_pos}")
            else:
                print(f"⚠️ Manque l'attribut 'pos' pour {u} ou {v}")
        else:
            print(f"⚠️ Bus introuvables pour {cable['name']}")
            
    print(f"\nTotal : {added} câbles connectés avec positions dynamiques.")


# Exécution
add_submarine_cables_with_midpoint(grid, 'submarine_cables.json')

In [ ]:
# ===========================================
# CALCULATE BASELINE PSI (POWER FLOW)
# ===========================================

print("Running Lanczos iterations...")
grid.iterate_qs()
grid.calculate_kappa()
psi_baseline = grid.calculate_psi_approx()
grid.apply_psi_to_graph(0)

print(f"✓ Completed {len(grid.q_snapshots)} iterations")

# Statistics
line_psi_abs = [abs(psi_baseline.get(l, 0)) for l in grid._lines]
print(f"\n=== BASELINE PSI STATISTICS ===")
print(f"  Max |ψ|: {max(line_psi_abs):.6f}")
print(f"  Mean |ψ|: {np.mean(line_psi_abs):.6f}")
print(f"  Median |ψ|: {np.median(line_psi_abs):.6f}")

---
# 🗺️ Partie 1 : Introduction du Réseau

Visualisation du réseau de transport haute tension européen avec la distribution des flux de puissance.

**Légende :**
- **Couleur des lignes** : Magnitude du flux (|ψ|) - Échelle Inferno (sombre = faible, lumineux = élevé)
- **Cercles verts** : Générateurs nets (production > consommation)
- **Cercles rouges** : Consommateurs nets (consommation > production)
- **Triangles oranges** : Nœuds de ligne avec valeur |ψ| (flux de puissance estimé)

**Estimation de puissance** : Pour convertir ψ en puissance, on utilise $P_{ligne} \approx |ψ| \times P_{ref}$ où $P_{ref}$ est la puissance totale injectée dans le réseau.

In [ ]:
# ===========================================
# FONCTION D'ESTIMATION DE PUISSANCE PAR LIGNE
# ===========================================

def estimate_line_power(G, psi_dict, bus_power):
    """
    Estimer la puissance réelle sur chaque ligne en utilisant
    la conservation de puissance aux nœuds.
    
    Pour chaque bus, on a : P_bus = Σ P_lignes_connectées (avec signe)
    On calcule un facteur de scaling local basé sur ψ et la puissance du bus.
    """
    line_power = {}
    scaling_factors = []
    
    # Pour chaque ligne, calculer la puissance estimée
    for line_node in G._lines:
        psi_val = psi_dict.get(line_node, 0)
        
        # Trouver les bus connectés à cette ligne
        neighbors = list(G.neighbors(line_node))
        bus_neighbors = [n for n in neighbors if G.nodes[n].get('type') == 'node']
        
        if len(bus_neighbors) == 2:
            bus0, bus1 = bus_neighbors
            bus0_id = str(bus0).replace('N_', '')
            bus1_id = str(bus1).replace('N_', '')
            
            p0 = bus_power.get(bus0_id, 0)
            p1 = bus_power.get(bus1_id, 0)
            
            # Le flux sur la ligne est proportionnel à |ψ|
            # et doit contribuer au bilan de puissance des deux bus
            # On utilise la moyenne des puissances absolues des bus comme référence
            avg_power = (abs(p0) + abs(p1)) / 2
            
            line_power[line_node] = abs(psi_val) * avg_power if avg_power > 0 else 0
        else:
            line_power[line_node] = 0
    
    # Normaliser par la somme des ψ entrant à chaque nœud
    # pour que le bilan soit cohérent
    for node in G._nodes:
        bus_id = str(node).replace('N_', '')
        p_bus = abs(bus_power.get(bus_id, 0))
        if p_bus < 1:
            continue
            
        # Somme des |ψ| sur les lignes connectées
        neighbors = list(G.neighbors(node))
        line_neighbors = [n for n in neighbors if G.nodes[n].get('type') == 'line']
        
        if not line_neighbors:
            continue
            
        sum_psi = sum(abs(psi_dict.get(ln, 0)) for ln in line_neighbors)
        
        if sum_psi > 0.001:
            # Facteur de scaling local
            local_scale = p_bus / sum_psi
            scaling_factors.append(local_scale)
    
    # Utiliser la médiane des facteurs de scaling (plus robuste)
    if scaling_factors:
        median_scale = np.median(scaling_factors)
    else:
        median_scale = 1000  # fallback
    
    # Recalculer avec le facteur médian
    for line_node in G._lines:
        psi_val = abs(psi_dict.get(line_node, 0))
        line_power[line_node] = psi_val * median_scale
    
    return line_power, median_scale


# Calculer les puissances estimées sur les lignes
line_power_estimates, SCALE_FACTOR = estimate_line_power(grid, psi_baseline, grid.bus_power)

# Statistiques
power_values = [p for p in line_power_estimates.values() if p > 0]
print(f"✓ Estimation de puissance par ligne (facteur d'échelle médian: {SCALE_FACTOR:.1f} MW)")
print(f"  Puissance max: {max(power_values):.0f} MW")
print(f"  Puissance moyenne: {np.mean(power_values):.0f} MW")
print(f"  Puissance médiane: {np.median(power_values):.0f} MW")

# ===========================================
# FONCTION DE VISUALISATION : RÉSEAU AVEC ÉCHELLE INFERNO
# ===========================================

# Définir l'échelle Inferno pour réutilisation
INFERNO_COLORSCALE = [
    [0.0, 'rgb(0,0,4)'],
    [0.1, 'rgb(40,11,84)'],
    [0.2, 'rgb(89,17,106)'],
    [0.3, 'rgb(128,29,98)'],
    [0.4, 'rgb(165,50,77)'],
    [0.5, 'rgb(197,85,62)'],
    [0.6, 'rgb(222,119,42)'],
    [0.7, 'rgb(241,157,21)'],
    [0.8, 'rgb(249,197,27)'],
    [0.9, 'rgb(252,229,48)'],
    [1.0, 'rgb(252,255,164)']
]

def plot_grid_baseline(G, psi_dict, bus_power, line_power, line_width_scale=5, node_size=12, title="Réseau Électrique Européen"):
    """
    Visualiser le réseau avec échelle Inferno pour les flux de puissance.
    Inclut les nœuds de ligne avec estimation de puissance réaliste.
    """
    # Valeurs psi des lignes
    line_psis = {l: abs(psi_dict.get(l, 0)) for l in G._lines}
    max_psi = max(line_psis.values()) if line_psis else 1
    max_power = max(line_power.values()) if line_power else 1
    
    # Créer des bins pour les couleurs (Inferno)
    num_bins = 25
    inferno_colors = [
        'rgb(0,0,4)', 'rgb(20,11,53)', 'rgb(57,15,97)', 'rgb(89,17,106)',
        'rgb(120,28,109)', 'rgb(149,44,99)', 'rgb(175,63,82)', 'rgb(197,85,62)',
        'rgb(215,110,41)', 'rgb(229,140,23)', 'rgb(238,173,15)', 'rgb(243,206,21)',
        'rgb(248,230,32)', 'rgb(252,255,164)'
    ]
    
    bins = {i: {'lat': [], 'lon': [], 
                'width': 1 + (i/num_bins) * line_width_scale,
                'color': inferno_colors[min(i * len(inferno_colors) // (num_bins + 1), len(inferno_colors)-1)]} 
            for i in range(num_bins + 1)}
    
    for u, v in G.edges():
        line_node = u if G.nodes[u].get('type') == 'line' else (v if G.nodes[v].get('type') == 'line' else None)
        if not line_node:
            continue
        
        psi_val = abs(psi_dict.get(line_node, 0))
        norm_psi = psi_val / max_psi if max_psi > 0 else 0
        bin_idx = min(int(norm_psi * num_bins), num_bins)
        
        lon0, lat0 = G.nodes[u]['pos']
        lon1, lat1 = G.nodes[v]['pos']
        bins[bin_idx]['lat'].extend([lat0, lat1, None])
        bins[bin_idx]['lon'].extend([lon0, lon1, None])
    
    edge_traces = [go.Scattermap(
        lat=d['lat'], lon=d['lon'], mode='lines',
        line=dict(width=d['width'], color=d['color']),
        hoverinfo='none', showlegend=False
    ) for d in bins.values() if d['lat']]
    
    # Traces des nœuds bus
    gen_lat, gen_lon, gen_sizes, gen_text = [], [], [], []
    load_lat, load_lon, load_sizes, load_text = [], [], [], []
    
    # Traces des nœuds de ligne avec couleur Inferno
    line_lat, line_lon, line_sizes, line_text, line_psi_values = [], [], [], [], []
    
    max_bus_power = max(abs(p) for p in bus_power.values()) if bus_power else 1
    
    for node, data in G.nodes(data=True):
        lon, lat = data['pos']
        
        if data.get('type') == 'node':
            bus_id = str(node).replace('N_', '')
            if bus_id not in bus_power:
                continue
            
            power = bus_power[bus_id]
            size = node_size * (0.5 + 0.8 * abs(power) / max_bus_power)
            hover = f"<b>Bus: {bus_id}</b><br>Puissance nette: {power:.0f} MW"
            
            if power > 0:
                gen_lat.append(lat); gen_lon.append(lon)
                gen_sizes.append(size); gen_text.append(hover)
            else:
                load_lat.append(lat); load_lon.append(lon)
                load_sizes.append(size); load_text.append(hover)
        
        # NŒUDS DE LIGNE - afficher ψ et puissance estimée
        elif data.get('type') == 'line':
            psi_val = abs(psi_dict.get(node, 0))
            power_est = line_power.get(node, 0)
            size = node_size * (0.4 + 0.8 * psi_val / max_psi)
            hover = f"<b>Ligne: {node}</b><br>|ψ|: {psi_val:.4f}<br>P estimée: {power_est:.0f} MW"
            line_lat.append(lat); line_lon.append(lon)
            line_sizes.append(size); line_text.append(hover)
            line_psi_values.append(psi_val)
    
    gen_trace = go.Scattermap(
        lat=gen_lat, lon=gen_lon, mode='markers', name='Générateur net (+)',
        text=gen_text, hoverinfo='text',
        marker=dict(size=gen_sizes, color='limegreen', symbol='circle', opacity=0.8)
    )
    
    load_trace = go.Scattermap(
        lat=load_lat, lon=load_lon, mode='markers', name='Consommateur net (-)',
        text=load_text, hoverinfo='text',
        marker=dict(size=load_sizes, color='red', symbol='circle', opacity=0.8)
    )
    
    # Trace des nœuds de ligne avec colorbar Inferno
    line_trace = go.Scattermap(
        lat=line_lat, lon=line_lon, mode='markers', name='Nœud de ligne',
        text=line_text, hoverinfo='text',
        marker=dict(
            size=line_sizes, 
            color=line_psi_values,
            colorscale='Inferno',
            cmin=0,
            cmax=max_psi,
            symbol='triangle', 
            opacity=0.8,
            colorbar=dict(
                title=dict(text='|ψ|', side='right'),
                x=1.02,
                y=0.5,
                len=0.6,
                thickness=15,
                tickformat='.2f'
            )
        )
    )
    
    fig = go.Figure(data=edge_traces + [gen_trace, load_trace, line_trace])
    
    fig.update_layout(
        title=f'{title}<br><sup>Max |ψ|: {max_psi:.4f} | Facteur d\'échelle: {SCALE_FACTOR:.0f} MW</sup>',
        margin=dict(b=0, l=0, r=80, t=60),
        width=1200, height=700,
        map=dict(style="basic", center=dict(lat=48, lon=8), zoom=3.8),
        #legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.9)")
    )
    
    return fig

print("✓ Fonctions de visualisation définies")

In [ ]:
# ===========================================
# AFFICHER LA CARTE DE RÉFÉRENCE
# ===========================================

fig_baseline = plot_grid_baseline(
    grid, psi_baseline, grid.bus_power, line_power_estimates,
    line_width_scale=7, node_size=15,
    title=""#"⚡ Réseau Électrique Européen - État de Référence (|ψ|)"
)
fig_baseline.show()

---
# ⚡ Partie 2 : Cadre de Simulation de Coupure de Ligne

Cette section définit les fonctions principales pour simuler les coupures de ligne en utilisant la **méthode de superposition**.

In [ ]:
# ===========================================
# FONCTIONS DE SIMULATION DE COUPURE DE LIGNE
# ===========================================

def get_line_info(line_id, pypsa_network, grid):
    """Obtenir les informations sur une ligne spécifique."""
    if line_id in pypsa_network.lines.index:
        row = pypsa_network.lines.loc[line_id]
        bus0, bus1 = row['bus0'], row['bus1']
        length = row.get('length', 0)
        
        # Obtenir les coordonnées
        x0, y0 = pypsa_network.buses.loc[bus0, ['x', 'y']]
        x1, y1 = pypsa_network.buses.loc[bus1, ['x', 'y']]
        
        return {
            'line_id': line_id,
            'bus0': bus0,
            'bus1': bus1,
            'length_km': length,
            'coords': ((x0, y0), (x1, y1))
        }
    return None


def calculate_dipole_psi(pypsa_network, bus_from, bus_to, grid_template):
    """
    Calculer ψ dipôle pour une paire source-puits donnée.
    Représente le motif de flux lors d'une injection de +1 à bus_from et -1 à bus_to.
    """
    # Créer un nouveau réseau pour le calcul du dipôle
    grid_dipole = EuropeanGrid(
        pypsa_network=pypsa_network, 
        q_N=len(pypsa_network.buses) * 2,
        ix=bus_from, ex=bus_to,
        iw=1, ew=-1,
        real_data=True,
        use_real_power=False  # Utiliser l'injection dipôle, pas la puissance réelle
    )
    
    grid_dipole.build_from_pypsa()
    
    # Filtrer au même composant connexe que le réseau principal
    largest_cc = max(nx.connected_components(grid_dipole), key=len)
    nodes_to_remove = set(grid_dipole.nodes()) - largest_cc
    grid_dipole.remove_nodes_from(nodes_to_remove)
    
    # Mettre à jour les listes internes
    grid_dipole._nodes = [n for n in grid_dipole._nodes if n in grid_dipole.nodes()]
    grid_dipole._lines = [l for l in grid_dipole._lines if l in grid_dipole.nodes()]
    
    # Exécuter les itérations de Lanczos
    grid_dipole.iterate_qs()
    grid_dipole.calculate_kappa()
    psi_dipole = grid_dipole.calculate_psi_approx()
    
    return psi_dipole, grid_dipole


def simulate_line_outage(line_id, psi_baseline, psi_dipole, grid):
    """
    Simuler une coupure de ligne par superposition :
    ψ_après = ψ_avant - I_coupé × ψ_dipôle
    """
    line_node = f"L_{line_id}"
    
    # Obtenir le courant traversant la ligne coupée
    I_cut = psi_baseline.get(line_node, 0)
    
    # Appliquer la superposition
    psi_after = {}
    for node in grid.nodes():
        if node == line_node:
            psi_after[node] = 0  # La ligne coupée n'a pas de flux
        else:
            psi_before = psi_baseline.get(node, 0)
            psi_dip = psi_dipole.get(node, 0)
            psi_after[node] = psi_before - I_cut * psi_dip * np.sqrt(2)
    
    # Calculer delta |ψ| (changement de flux absolu)
    psi_delta = {}
    for node in grid.nodes():
        if node == line_node:
            continue
        before_abs = abs(psi_baseline.get(node, 0))
        after_abs = abs(psi_after.get(node, 0))
        psi_delta[node] = after_abs - before_abs
    
    return psi_after, psi_delta, I_cut


def create_results_table(line_id, psi_baseline, psi_delta, grid, pypsa_network, scale_factor, top_n=15):
    """Créer un tableau récapitulatif des lignes les plus affectées."""
    rows = []
    
    for line_node in grid._lines:
        if line_node == f"L_{line_id}":
            continue  # Ignorer la ligne coupée
            
        lid = line_node.replace('L_', '')
        delta = psi_delta.get(line_node, 0)
        psi_before = abs(psi_baseline.get(line_node, 0))
        psi_after = psi_before + delta
        
        # Puissance estimée avec le facteur d'échelle médian
        P_before = psi_before * scale_factor
        P_after = psi_after * scale_factor
        delta_P = delta * scale_factor
        
        # Obtenir les infos des bus depuis pypsa
        if lid in pypsa_network.lines.index:
            row_info = pypsa_network.lines.loc[lid]
            bus0, bus1 = row_info['bus0'], row_info['bus1']
        else:
            bus0, bus1 = "?", "?"
        
        rows.append({
            'Ligne': lid,
            'De': bus0,
            'Vers': bus1,
            '|ψ|_avant': psi_before,
            '|ψ|_après': psi_after,
            'Δ|ψ|': delta,
            'P_avant (MW)': P_before,
            'P_après (MW)': P_after,
            'ΔP (MW)': delta_P,
            'Variation %': (delta / psi_before * 100) if psi_before > 0.0001 else 0
        })
    
    df = pd.DataFrame(rows)
    df = df.sort_values('Δ|ψ|', ascending=False).reset_index(drop=True)
    
    return df.head(top_n)


print("✓ Fonctions de simulation définies")

In [ ]:
# ===========================================
# FONCTIONS DE VISUALISATION POUR L'ANALYSE DE COUPURE
# ===========================================

# Colorscale divergente pour Δψ (bleu = diminution, rouge = augmentation)
DIVERGING_COLORSCALE = [
    [0.0, 'rgb(33,102,172)'],    # Bleu foncé
    [0.25, 'rgb(103,169,207)'],  # Bleu clair
    [0.5, 'rgb(247,247,247)'],   # Blanc
    [0.75, 'rgb(239,138,98)'],   # Orange/Rouge clair
    [1.0, 'rgb(178,24,43)']      # Rouge foncé
]

def plot_delta_psi_map(G, psi_baseline, psi_delta, bus_power, cut_line_id, cut_info, scale_factor,
                        line_width_scale=5, node_size=12, width=1200, height=700):
    """
    Visualiser Δ|ψ| après coupure de ligne.
    Utilise l'échelle Inferno pour visualiser l'amplitude des changements.
    Inclut les nœuds de ligne avec estimation de puissance.
    """
    line_node = f"L_{cut_line_id}"
    
    # Obtenir max delta pour normalisation (valeur absolue)
    line_deltas = {l: abs(psi_delta.get(l, 0)) for l in G._lines if l != line_node}
    max_delta = max(line_deltas.values()) if line_deltas else 1
    
    # Créer des bins pour les couleurs (Inferno)
    num_bins = 25
    inferno_colors = [
        'rgb(0,0,4)', 'rgb(20,11,53)', 'rgb(57,15,97)', 'rgb(89,17,106)',
        'rgb(120,28,109)', 'rgb(149,44,99)', 'rgb(175,63,82)', 'rgb(197,85,62)',
        'rgb(215,110,41)', 'rgb(229,140,23)', 'rgb(238,173,15)', 'rgb(243,206,21)',
        'rgb(248,230,32)', 'rgb(252,255,164)'
    ]
    
    bins = {i: {'lat': [], 'lon': [], 
                'width': 1 + (i/num_bins) * line_width_scale,
                'color': inferno_colors[min(i * len(inferno_colors) // (num_bins + 1), len(inferno_colors)-1)]} 
            for i in range(num_bins + 1)}
    
    for u, v in G.edges():
        line_node_edge = u if G.nodes[u].get('type') == 'line' else (v if G.nodes[v].get('type') == 'line' else None)
        if not line_node_edge or line_node_edge == line_node:
            continue
        
        delta_val = abs(psi_delta.get(line_node_edge, 0))
        norm_delta = delta_val / max_delta if max_delta > 0 else 0
        bin_idx = min(int(norm_delta * num_bins), num_bins)
        
        lon0, lat0 = G.nodes[u]['pos']
        lon1, lat1 = G.nodes[v]['pos']
        bins[bin_idx]['lat'].extend([lat0, lat1, None])
        bins[bin_idx]['lon'].extend([lon0, lon1, None])
    
    edge_traces = [go.Scattermap(
        lat=d['lat'], lon=d['lon'], mode='lines',
        line=dict(width=d['width'], color=d['color']),
        hoverinfo='none', showlegend=False
    ) for d in bins.values() if d['lat']]
    
    # Traces des nœuds bus
    gen_lat, gen_lon, gen_sizes, gen_text = [], [], [], []
    load_lat, load_lon, load_sizes, load_text = [], [], [], []
    
    # Traces des nœuds de ligne avec valeurs delta
    line_lat, line_lon, line_sizes, line_text, line_delta_abs_values = [], [], [], [], []
    
    max_power = max(abs(p) for p in bus_power.values()) if bus_power else 1
    
    for node, data in G.nodes(data=True):
        lon, lat = data['pos']
        
        if data.get('type') == 'node':
            bus_id = str(node).replace('N_', '')
            if bus_id not in bus_power:
                continue
            
            power = bus_power[bus_id]
            size = node_size * (0.5 + 0.8 * abs(power) / max_power)
            hover = f"<b>Bus: {bus_id}</b><br>Puissance nette: {power:.0f} MW"
            
            if power > 0:
                gen_lat.append(lat); gen_lon.append(lon)
                gen_sizes.append(size); gen_text.append(hover)
            else:
                load_lat.append(lat); load_lon.append(lon)
                load_sizes.append(size); load_text.append(hover)
        
        # NŒUDS DE LIGNE avec Δψ
        elif data.get('type') == 'line' and node != line_node:
            delta_val = psi_delta.get(node, 0)
            psi_before = abs(psi_baseline.get(node, 0))
            psi_after = psi_before + delta_val
            
            # Utiliser le facteur d'échelle médian pour l'estimation de puissance
            P_before = psi_before * scale_factor
            P_after = psi_after * scale_factor
            delta_P = delta_val * scale_factor
            
            size = node_size * (0.4 + 0.8 * abs(delta_val) / max_delta)
            
            hover = (f"<b>Ligne: {node}</b><br>"
                     f"|ψ|_avant: {psi_before:.4f}<br>"
                     f"|ψ|_après: {psi_after:.4f}<br>"
                     f"Δ|ψ|: {delta_val:+.4f}<br>"
                     f"<b>P_avant: {P_before:.0f} MW</b><br>"
                     f"<b>P_après: {P_after:.0f} MW</b><br>"
                     f"<b>ΔP: {delta_P:+.0f} MW</b>")
            
            line_lat.append(lat); line_lon.append(lon)
            line_sizes.append(size)
            line_text.append(hover)
            line_delta_abs_values.append(abs(delta_val))
    
    gen_trace = go.Scattermap(
        lat=gen_lat, lon=gen_lon, mode='markers', name='Générateur net',
        text=gen_text, hoverinfo='text',
        marker=dict(size=gen_sizes, color='limegreen', symbol='circle', opacity=0.7)
    )
    
    load_trace = go.Scattermap(
        lat=load_lat, lon=load_lon, mode='markers', name='Consommateur net',
        text=load_text, hoverinfo='text',
        marker=dict(size=load_sizes, color='red', symbol='circle', opacity=0.7)
    )
    
    # Nœuds de ligne avec colorbar Inferno
    line_trace = go.Scattermap(
        lat=line_lat, lon=line_lon, mode='markers', name='Nœud de ligne',
        text=line_text, hoverinfo='text',
        marker=dict(
            size=line_sizes, 
            color=line_delta_abs_values,
            colorscale='Inferno',
            cmin=0,
            cmax=max_delta,
            symbol='triangle', 
            opacity=0.8,
            colorbar=dict(
                title=dict(text='|Δψ|', side='right'),
                x=1.02,
                y=0.5,
                len=0.6,
                thickness=15,
                tickformat='.3f'
            )
        )
    )
    
    # Marquer la ligne coupée avec un X
    cut_traces = []
    if cut_info:
        (x0, y0), (x1, y1) = cut_info['coords']
        # Marqueur X au milieu de la ligne coupée
        cut_marker_trace = go.Scattermap(
            lat=[(y0+y1)/2], lon=[(x0+x1)/2], mode='markers',
            marker=dict(size=25, color='black', symbol='x'),
            name=f'COUPÉE: Ligne {cut_line_id}',
            text=[f"<b>LIGNE COUPÉE {cut_line_id}</b><br>{cut_info['bus0']} ↔ {cut_info['bus1']}"],
            hoverinfo='text',
            showlegend=True
        )
        cut_traces.append(cut_marker_trace)
    
    fig = go.Figure(data=edge_traces + [gen_trace, load_trace, line_trace] + cut_traces)
    
    fig.update_layout(
        title="",#f'⚡ Redistribution des Flux (Δ|ψ|) - Ligne {cut_line_id} Coupée<br><sup style="color:red">■ Charge augmentée</sup> | <sup style="color:blue">■ Charge diminuée</sup> | Max |Δψ|: {max_delta:.4f}',
        margin=dict(b=0, l=0, r=80, t=80),
        width=width, height=height,
        map=dict(style="basic", center=dict(lat=48, lon=8), zoom=3.8),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.9)")
    )
    
    return fig


def plot_psi_after_map(G, psi_after, bus_power, cut_line_id, cut_info, scale_factor,
                        line_width_scale=5, node_size=12, width=1200, height=700):
    """
    Visualiser |ψ| après coupure de ligne.
    Utilise l'échelle Inferno pour visualiser les flux absolus après la coupure.
    """
    line_node = f"L_{cut_line_id}"
    
    # Valeurs psi des lignes (après coupure)
    line_psis = {l: abs(psi_after.get(l, 0)) for l in G._lines if l != line_node}
    max_psi = max(line_psis.values()) if line_psis else 1
    
    # Créer des bins pour les couleurs (Inferno)
    num_bins = 25
    inferno_colors = [
        'rgb(0,0,4)', 'rgb(20,11,53)', 'rgb(57,15,97)', 'rgb(89,17,106)',
        'rgb(120,28,109)', 'rgb(149,44,99)', 'rgb(175,63,82)', 'rgb(197,85,62)',
        'rgb(215,110,41)', 'rgb(229,140,23)', 'rgb(238,173,15)', 'rgb(243,206,21)',
        'rgb(248,230,32)', 'rgb(252,255,164)'
    ]
    
    bins = {i: {'lat': [], 'lon': [], 
                'width': 1 + (i/num_bins) * line_width_scale,
                'color': inferno_colors[min(i * len(inferno_colors) // (num_bins + 1), len(inferno_colors)-1)]} 
            for i in range(num_bins + 1)}
    
    for u, v in G.edges():
        line_node_edge = u if G.nodes[u].get('type') == 'line' else (v if G.nodes[v].get('type') == 'line' else None)
        if not line_node_edge or line_node_edge == line_node:
            continue
        
        psi_val = abs(psi_after.get(line_node_edge, 0))
        norm_psi = psi_val / max_psi if max_psi > 0 else 0
        bin_idx = min(int(norm_psi * num_bins), num_bins)
        
        lon0, lat0 = G.nodes[u]['pos']
        lon1, lat1 = G.nodes[v]['pos']
        bins[bin_idx]['lat'].extend([lat0, lat1, None])
        bins[bin_idx]['lon'].extend([lon0, lon1, None])
    
    edge_traces = [go.Scattermap(
        lat=d['lat'], lon=d['lon'], mode='lines',
        line=dict(width=d['width'], color=d['color']),
        hoverinfo='none', showlegend=False
    ) for d in bins.values() if d['lat']]
    
    # Traces des nœuds bus
    gen_lat, gen_lon, gen_sizes, gen_text = [], [], [], []
    load_lat, load_lon, load_sizes, load_text = [], [], [], []
    
    # Traces des nœuds de ligne avec couleur Inferno
    line_lat, line_lon, line_sizes, line_text, line_psi_values = [], [], [], [], []
    
    max_bus_power = max(abs(p) for p in bus_power.values()) if bus_power else 1
    
    for node, data in G.nodes(data=True):
        lon, lat = data['pos']
        
        if data.get('type') == 'node':
            bus_id = str(node).replace('N_', '')
            if bus_id not in bus_power:
                continue
            
            power = bus_power[bus_id]
            size = node_size * (0.5 + 0.8 * abs(power) / max_bus_power)
            hover = f"<b>Bus: {bus_id}</b><br>Puissance nette: {power:.0f} MW"
            
            if power > 0:
                gen_lat.append(lat); gen_lon.append(lon)
                gen_sizes.append(size); gen_text.append(hover)
            else:
                load_lat.append(lat); load_lon.append(lon)
                load_sizes.append(size); load_text.append(hover)
        
        # NŒUDS DE LIGNE - afficher ψ et puissance estimée après coupure
        elif data.get('type') == 'line' and node != line_node:
            psi_val = abs(psi_after.get(node, 0))
            power_est = psi_val * scale_factor
            size = node_size * (0.4 + 0.8 * psi_val / max_psi)
            hover = f"<b>Ligne: {node}</b><br>|ψ|_après: {psi_val:.4f}<br>P estimée: {power_est:.0f} MW"
            line_lat.append(lat); line_lon.append(lon)
            line_sizes.append(size); line_text.append(hover)
            line_psi_values.append(psi_val)
    
    gen_trace = go.Scattermap(
        lat=gen_lat, lon=gen_lon, mode='markers', name='Générateur net (+)',
        text=gen_text, hoverinfo='text',
        marker=dict(size=gen_sizes, color='limegreen', symbol='circle', opacity=0.8)
    )
    
    load_trace = go.Scattermap(
        lat=load_lat, lon=load_lon, mode='markers', name='Consommateur net (-)',
        text=load_text, hoverinfo='text',
        marker=dict(size=load_sizes, color='red', symbol='circle', opacity=0.8)
    )
    
    # Trace des nœuds de ligne avec colorbar Inferno
    line_trace = go.Scattermap(
        lat=line_lat, lon=line_lon, mode='markers', name='Nœud de ligne',
        text=line_text, hoverinfo='text',
        marker=dict(
            size=line_sizes, 
            color=line_psi_values,
            colorscale='Inferno',
            cmin=0,
            cmax=max_psi,
            symbol='triangle', 
            opacity=0.8,
            colorbar=dict(
                title=dict(text='|ψ|', side='right'),
                x=1.02,
                y=0.5,
                len=0.6,
                thickness=15,
                tickformat='.3f'
            )
        )
    )

    
    fig = go.Figure(data=edge_traces + [gen_trace, load_trace, line_trace])
    
    fig.update_layout(

        title="",#f'⚡ Flux de Puissance Après Coupure (|ψ|_après) - Ligne {cut_line_id} Coupée<br><sup>Max |ψ|_après: {max_psi:.4f} | Facteur d\'échelle: {scale_factor:.0f} MW</sup>',
        margin=dict(b=0, l=0, r=80, t=80),
        width=width, height=height,
        map=dict(style="basic", center=dict(lat=48, lon=8), zoom=3.8),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.9)")

    )

    
    return fig
print("✓ Fonctions de visualisation pour l'analyse de coupure définies")

---
# 🔴 Étude de Cas 1 : Coupure de la Ligne Lyon-Grenoble (Ligne 380)

**La ligne 380** relie deux nœuds majeurs de la région Rhône-Alpes :
- **FR1 1** (Région lyonnaise) : Coordonnées (4.99°E, 45.69°N)
- **FR1 20** (Région grenobloise) : Coordonnées (5.95°E, 45.56°N)

C'est une **ligne interne française** reliant deux grandes régions industrielles.

In [ ]:
# ===========================================
# CAS 1 : LIGNE LYON-GRENOBLE (378)
# ===========================================

LINE_1 = "380"  # Lyon (FR1 1) ↔ Grenoble (FR1 20)

# Obtenir les informations de la ligne
line_info_1 = get_line_info(LINE_1, n, grid)
print(f"=== CAS 1 : LIGNE {LINE_1} ===")
print(f"Trajet : {line_info_1['bus0']} (Lyon) ↔ {line_info_1['bus1']} (Grenoble)")
print(f"Longueur : {line_info_1['length_km']:.1f} km")
print(f"|ψ| de référence : {abs(psi_baseline.get(f'L_{LINE_1}', 0)):.6f}")

In [ ]:
# ===========================================
# CALCULER ψ DIPÔLE POUR LYON-GRENOBLE
# ===========================================

print("Calcul de ψ dipôle pour les extrémités de la ligne Lyon-Grenoble...")
psi_dipole_1, grid_dipole_1 = calculate_dipole_psi(n, line_info_1['bus0'], line_info_1['bus1'], grid)
print(f"✓ Calcul du dipôle terminé")

In [ ]:
# ===========================================
# SIMULER LA COUPURE DE LA LIGNE 378
# ===========================================

psi_after_1, psi_delta_1, I_cut_1 = simulate_line_outage(LINE_1, psi_baseline, psi_dipole_1, grid)

print(f"\n{'='*60}")
print(f"RÉSULTATS DE SIMULATION DE COUPURE : Ligne {LINE_1}")
print(f"{'='*60}")
print(f"I_coupé (flux avant coupure) : {I_cut_1:.6f}")
print(f"Puissance coupée estimée : {abs(I_cut_1) * SCALE_FACTOR:.0f} MW")

# Statistiques récapitulatives
delta_lines_1 = {l: psi_delta_1.get(l, 0) for l in grid._lines if l != f"L_{LINE_1}"}
print(f"\nStatistiques d'impact :")
print(f"  Max AUGMENTATION (Δ|ψ|) : {max(delta_lines_1.values()):.6f} → {max(delta_lines_1.values()) * SCALE_FACTOR:.0f} MW")
print(f"  Max DIMINUTION (Δ|ψ|) : {min(delta_lines_1.values()):.6f} → {min(delta_lines_1.values()) * SCALE_FACTOR:.0f} MW")
print(f"  Moyenne |Δ|ψ|| : {np.mean([abs(d) for d in delta_lines_1.values()]):.6f}")

In [ ]:
# ===========================================
# TABLEAU DES RÉSULTATS : COUPURE LYON-GRENOBLE
# ===========================================

table_1 = create_results_table(LINE_1, psi_baseline, psi_delta_1, grid, n, SCALE_FACTOR, top_n=15)

print(f"\n{'='*90}")
print(f"TOP 15 LIGNES LES PLUS AFFECTÉES - Coupure Ligne {LINE_1} (Lyon-Grenoble)")
print(f"{'='*90}")

# Styliser le tableau
styled_table_1 = table_1.style.format({
    '|ψ|_avant': '{:.6f}',
    '|ψ|_après': '{:.6f}',
    'Δ|ψ|': '{:+.6f}',
    'P_avant (MW)': '{:.0f}',
    'P_après (MW)': '{:.0f}',
    'ΔP (MW)': '{:+.0f}',
    'Variation %': '{:+.1f}%'
}).background_gradient(subset=['Δ|ψ|'], cmap='RdBu_r', vmin=-table_1['Δ|ψ|'].abs().max(), vmax=table_1['Δ|ψ|'].abs().max())

display(styled_table_1)

In [ ]:
# ===========================================
# CARTE : DELTA PSI - COUPURE LYON-GRENOBLE
# ===========================================

fig_delta_1 = plot_delta_psi_map(grid, psi_baseline, psi_delta_1, grid.bus_power, 
                                  LINE_1, line_info_1, SCALE_FACTOR, line_width_scale=14, node_size=25)
fig_delta_1.show()

In [ ]:
# ===========================================
# CARTE : PSI APRÈS COUPURE - CAS 1 (LYON-GRENOBLE)
# ===========================================

fig_after_1 = plot_psi_after_map(grid, psi_after_1, grid.bus_power, 
                                  LINE_1, line_info_1, SCALE_FACTOR, line_width_scale=14, node_size=30)
fig_after_1.show()

---
# 🔵 Étude de Cas 2 : Coupure de la Ligne Perpignan-Gérone (Ligne 340)

**La ligne 340** est une **interconnexion transfrontalière critique** entre la France et l'Espagne :
- **FR1 34** (Région de Perpignan) : Coordonnées (2.80°E, 43.06°N)
- **ES1 30** (Région de Gérone) : Coordonnées (2.77°E, 42.01°N)

Cette ligne traverse les **Pyrénées** et représente l'un des rares points d'interconnexion entre les réseaux français et espagnol.

In [ ]:
# ===========================================
# CAS 2 : LIGNE PERPIGNAN-GÉRONE (340)
# ===========================================

LINE_2 = "340"  # Perpignan (FR1 34) ↔ Gérone (ES1 30)

# Obtenir les informations de la ligne
line_info_2 = get_line_info(LINE_2, n, grid)
print(f"=== CAS 2 : LIGNE {LINE_2} ===")
print(f"Trajet : {line_info_2['bus0']} (Gérone, ES) ↔ {line_info_2['bus1']} (Perpignan, FR)")
print(f"Longueur : {line_info_2['length_km']:.1f} km")
print(f"|ψ| de référence : {abs(psi_baseline.get(f'L_{LINE_2}', 0)):.6f}")


In [ ]:
# ===========================================
# CALCULER ψ DIPÔLE POUR PERPIGNAN-GÉRONE
# ===========================================

print("Calcul de ψ dipôle pour les extrémités de la ligne Perpignan-Gérone...")
psi_dipole_2, grid_dipole_2 = calculate_dipole_psi(n, line_info_2['bus0'], line_info_2['bus1'], grid)
print(f"✓ Calcul du dipôle terminé")

In [ ]:
# ===========================================
# SIMULER LA COUPURE DE LA LIGNE 340
# ===========================================

psi_after_2, psi_delta_2, I_cut_2 = simulate_line_outage(LINE_2, psi_baseline, psi_dipole_2, grid)

print(f"\n{'='*60}")
print(f"RÉSULTATS DE SIMULATION DE COUPURE : Ligne {LINE_2}")
print(f"{'='*60}")
print(f"I_coupé (flux avant coupure) : {I_cut_2:.6f}")
print(f"Puissance coupée estimée : {abs(I_cut_2) * SCALE_FACTOR:.0f} MW")

# Statistiques récapitulatives
delta_lines_2 = {l: psi_delta_2.get(l, 0) for l in grid._lines if l != f"L_{LINE_2}"}
print(f"\nStatistiques d'impact :")
print(f"  Max AUGMENTATION (Δ|ψ|) : {max(delta_lines_2.values()):.6f} → {max(delta_lines_2.values()) * SCALE_FACTOR:.0f} MW")
print(f"  Max DIMINUTION (Δ|ψ|) : {min(delta_lines_2.values()):.6f} → {min(delta_lines_2.values()) * SCALE_FACTOR:.0f} MW")
print(f"  Moyenne |Δ|ψ|| : {np.mean([abs(d) for d in delta_lines_2.values()]):.6f}")

In [ ]:
# ===========================================
# TABLEAU DES RÉSULTATS - CAS 2
# ===========================================

# Préparer les données des lignes

table_2 = create_results_table(LINE_2, psi_baseline, psi_delta_2, grid, n, SCALE_FACTOR, top_n=15)

print(f"\n{'='*90}")
print(f"TOP 15 LIGNES LES PLUS AFFECTÉES - Coupure Ligne {LINE_2} (Perpignan-Girona)")
print(f"{'='*90}")

# Styliser le tableau
styled_table_2 = table_2.style.format({
    '|ψ|_avant': '{:.6f}',
    '|ψ|_après': '{:.6f}',
    'Δ|ψ|': '{:+.6f}',
    'P_avant (MW)': '{:.0f}',
    'P_après (MW)': '{:.0f}',
    'ΔP (MW)': '{:+.0f}',
    'Variation %': '{:+.1f}%'
}).background_gradient(subset=['Δ|ψ|'], cmap='RdBu_r', vmin=-table_2['Δ|ψ|'].abs().max(), vmax=table_2['Δ|ψ|'].abs().max())

display(styled_table_2)

In [ ]:
# ===========================================
# CARTE DU DELTA PSI - CAS 2
# ===========================================

fig_delta_2 = plot_delta_psi_map(
    grid, 
    psi_baseline,
    psi_delta_2, 
    grid.bus_power,
    LINE_2,
    line_info_2,
    SCALE_FACTOR,
    line_width_scale=10,
    node_size=30
)
fig_delta_2.show()

In [ ]:
# ===========================================
# CARTE : PSI APRÈS COUPURE - CAS 2 (PERPIGNAN-GÉRONE)
# ===========================================

fig_after_2 = plot_psi_after_map(grid, psi_after_2, grid.bus_power, 
                                  LINE_2, line_info_2, SCALE_FACTOR, line_width_scale=14, node_size=25)
fig_after_2.show()

---
## Partie 4 : Analyse Comparative

Comparons l'impact des deux coupures de ligne côte à côte :
- **Cas 1** : Ligne intérieure française (Lyon ↔ Grenoble)
- **Cas 2** : Ligne transfrontalière France-Espagne (Perpignan ↔ Girona)

In [ ]:
# ===========================================
# TABLEAU COMPARATIF
# ===========================================

comparison_data = {
    'Métrique': [
        'ID de la ligne coupée',
        'Type de connexion',
        'Longueur de ligne (km)',
        'Flux coupé |I_coupé|',
        'Puissance coupée (MW)',
        'Max Δ|ψ| positif',
        'Max Δ|ψ| négatif',
        'Max ΔP positif (MW)',
        'Max ΔP négatif (MW)',
        'Moyenne |Δ|ψ||'
    ],
    'Cas 1 : Lyon-Grenoble': [
        LINE_1,
        'Intérieure (FR-FR)',
        f"{line_info_1['length_km']:.1f}",
        f"{abs(I_cut_1):.6f}",
        f"{abs(I_cut_1) * SCALE_FACTOR:.0f}",
        f"{max(delta_lines_1.values()):.6f}",
        f"{min(delta_lines_1.values()):.6f}",
        f"{max(delta_lines_1.values()) * SCALE_FACTOR:.0f}",
        f"{min(delta_lines_1.values()) * SCALE_FACTOR:.0f}",
        f"{np.mean([abs(d) for d in delta_lines_1.values()]):.6f}"
    ],
    'Cas 2 : Perpignan-Girona': [
        LINE_2,
        'Transfrontalière (FR-ES)',
        f"{line_info_2['length_km']:.1f}",
        f"{abs(I_cut_2):.6f}",
        f"{abs(I_cut_2) * SCALE_FACTOR:.0f}",
        f"{max(delta_lines_2.values()):.6f}",
        f"{min(delta_lines_2.values()):.6f}",
        f"{max(delta_lines_2.values()) * SCALE_FACTOR:.0f}",
        f"{min(delta_lines_2.values()) * SCALE_FACTOR:.0f}",
        f"{np.mean([abs(d) for d in delta_lines_2.values()]):.6f}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("TABLEAU COMPARATIF : Analyse d'Impact des Coupures de Ligne")
print("="*100)
display(df_comparison)

---
## Conclusion

### Principaux Résultats

Ce notebook démontre l'application de l'approche par superposition pour l'analyse de contingence N-1 sur le réseau électrique européen. Les points clés comprennent :

1. **Validation de la Méthode** : L'état de base ψ fournit une représentation cohérente des flux de puissance sur le réseau.

2. **Études de Cas** :
   - La coupure de la ligne **Lyon-Grenoble** (intérieure française) affecte principalement les lignes environnantes dans la région sud-est de la France
   - La coupure de la ligne **Perpignan-Girona** (transfrontalière) a un impact plus large sur les interconnexions entre la France et l'Espagne

3. **Redistribution des Flux** : La formule de superposition $\psi_{\text{après}} = \psi_{\text{avant}} - I_{\text{coupé}} \cdot \psi_{\text{dipôle}} \cdot \sqrt{2}$ capture efficacement la redistribution du flux après une coupure de ligne.

4. **Estimation de Puissance** : La puissance sur chaque ligne est estimée en utilisant un facteur d'échelle médian calculé à partir du bilan de puissance aux nœuds :
   $$P_{\text{ligne}} = |\psi| \times S_{\text{médian}}$$
   où $S_{\text{médian}}$ est dérivé de la relation $P_{\text{bus}} = \sum_{\text{lignes connectées}} |\psi|$

### Travaux Futurs

- Valider avec des résultats de load flow AC
- Incorporer les limites de capacité des lignes
